In [3]:
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tqdm
%matplotlib notebook
from IPython import display
import pandas as pd
import sys, subprocess
from PIL import Image
import pickle
from armcodlib import *
import keras.backend as K
import os
import cv2
from glob import glob

Using TensorFlow backend.


In [11]:
NB_POSTURE = 50
NB_COMMAND = 100
NB_DATA = NB_POSTURE*NB_COMMAND
BATCH_SIZE = 100
TEST_BUF = 1000
IMG_SIZE = 64
DIMS = (IMG_SIZE, IMG_SIZE,2)
N_TRAIN_BATCHES =int(NB_DATA/BATCH_SIZE)
N_TEST_BATCHES = int(TEST_BUF/BATCH_SIZE)

train_posture_before, train_posture_after, train_command, train_position_before, train_position_after  = create_random_data(NB_POSTURE, NB_COMMAND, "train")
train_images = load_and_process_images(NB_DATA, "train")
#test_posture_before, test_posture_after, test_command, test_position_before, test_position_after = create_random_data(10,10, "test")
#test_images = load_and_process_images(100, "test")

train_dataset = (
    tf.data.Dataset.from_tensor_slices((train_images[:,:,:,],train_images[:,:,:,1:]))
    .repeat(10)
    .shuffle(NB_DATA)
    .batch(BATCH_SIZE)
)

train_dataset_x = (
    tf.data.Dataset.from_tensor_slices((train_images[:,:,:,],train_images[:,:,:,:0]))
    .repeat(10)
    .shuffle(NB_DATA)
    .batch(BATCH_SIZE)
)

#test_dataset = (
#    tf.data.Dataset.from_tensor_slices((test_images, test_images[:,:,:,1:]))
#    .shuffle(TEST_BUF)
#    .batch(BATCH_SIZE)
#)

visu_dataset = (
    tf.data.Dataset.from_tensor_slices((train_images[:,:,:,]))
    .repeat(10)
    .shuffle(NB_DATA)
)

100%|██████████| 5000/5000 [00:17<00:00, 281.96it/s]


In [12]:

def build_conv2D_pointwise_encoder(custom_shape= INPUT_ENCODER_SHAPE):

    inputs = tf.keras.Input(shape=custom_shape, name='encoder_input')

    x = tf.keras.layers.Lambda(lambda x: x[:, :, :, 0])(inputs)
    x = tf.keras.layers.Reshape((IMG_SIZE, IMG_SIZE, 1))(x)

    y = tf.keras.layers.Lambda(lambda x: x[:, :, :, 1])(inputs)
    y = tf.keras.layers.Reshape((IMG_SIZE, IMG_SIZE, 1))(y)

    #fx = tf.keras.layers.Flatten()(x)
    fx = tf.keras.layers.Conv2D(filters=LATENT_DIM, kernel_size=3, strides=(
        2, 2), activation='relu', name='conv_x_1')(x)
    fx = tf.keras.layers.Conv2D(filters=LATENT_DIM * 2, kernel_size=3,
                                strides=(2, 2), activation='relu', name='conv_x_2')(fx)
    fx = tf.keras.layers.Flatten()(fx)
    #fx = tf.keras.layers.Dense(units=15*15*64, name = 'latent_fx1')(fx)
    fx = tf.keras.layers.Dense(LATENT_DIM, name='latent_fx2')(fx)
    fx = tf.keras.layers.Reshape((LATENT_DIM, 1,))(fx)

    #fy = tf.keras.layers.Flatten()(y)
    fy = tf.keras.layers.Conv2D(filters=LATENT_DIM, kernel_size=3, strides=(
        2, 2), activation='relu', name='conv_y_1')(y)
    fy = tf.keras.layers.Conv2D(filters=LATENT_DIM * 2, kernel_size=3,
                                strides=(2, 2), activation='relu', name='conv_y_2')(fy)
    fy = tf.keras.layers.Flatten()(fy)
    #fy = tf.keras.layers.Dense(units=15*15*64, name = 'latent_fy1')(fy)
    fy = tf.keras.layers.Dense(LATENT_DIM, name='latent_fy2')(fy)
    fy = tf.keras.layers.Reshape((LATENT_DIM,1,))(fy)

    matmul = tf.keras.layers.Multiply()([fx, fy])

    fh = tf.keras.layers.Flatten()(matmul)
    fh = tf.keras.layers.Dense(LATENT_DIM, name='latent_fh1')(fh)
    #fh = tf.keras.layers.Dense(LATENT_DIM, name = 'latent_fh2')(fh)
    #fh = tf.keras.layers.Dense(LATENT_DIM, name = 'latent_fh3')(fh)

    fx = tf.keras.layers.Reshape((1, LATENT_DIM,))(fx)
    fh = tf.keras.layers.Reshape((1, LATENT_DIM,))(fh)

    outputs = tf.keras.layers.Concatenate()([fx, fh])
    encoder = tf.keras.Model(
        inputs=inputs, outputs=outputs, name='encoder_model')

    return encoder

    
def build_dense_pointwise_decoder():

    inputs = tf.keras.Input(shape=INPUT_DECODER_SHAPE, name='decoder_input')

    fx = tf.keras.layers.Lambda(lambda x: x[:, :, :LATENT_DIM])(inputs)
    fx = tf.keras.layers.Reshape((LATENT_DIM, 1,))(fx)

    fh = tf.keras.layers.Lambda(lambda x: x[:, :, LATENT_DIM:])(inputs)
    fh = tf.keras.layers.Reshape((LATENT_DIM, 1,))(fh)

    fh = tf.keras.layers.Flatten()(fh)
    fh = tf.keras.layers.Dense(LATENT_DIM, name='latent_dec_fh1')(fh)
    fh = tf.keras.layers.Dense(LATENT_DIM, name='latent_dec_fh2')(fh)
    fh = tf.keras.layers.Dense(LATENT_DIM, name='latent_dec_fh3')(fh)
    fh = tf.keras.layers.Reshape((LATENT_DIM,1,))(fh)

    matmul = tf.keras.layers.Multiply()([fx, fh])

    fy = tf.keras.layers.Flatten()(matmul)
    fy = tf.keras.layers.Dense(LATENT_DIM, name='latent_dec_fy1')(fy)
    #fy = tf.keras.layers.Dense(LATENT_DIM, name = 'latent_dec_fy2')(fy)
    #fy = tf.keras.layers.Dense(LATENT_DIM, name = 'latent_dec_fy3')(fy)

    y = tf.keras.layers.Dense(
        IMG_SIZE * IMG_SIZE, activation='relu', name='y_recon')(fy)
    outputs = tf.keras.layers.Reshape((IMG_SIZE, IMG_SIZE, 1))(y)

    decoder = tf.keras.Model(
        inputs=inputs, outputs=outputs, name='decoder_model')

    return decoder


In [13]:
encoder_conv = build_conv2D_pointwise_encoder()
decoder_dense = build_dense_pointwise_decoder()
optimizer = tf.keras.optimizers.Adam(1e-3)

autoencoder_conv = tf.keras.Model(encoder_conv.input, decoder_dense(encoder_conv(encoder_conv.input)), name = "autoencoder")

autoencoder_conv.compile(optimizer = 'adam', 
             loss='mse',
             metrics=['accuracy','kullback_leibler_divergence'])


In [14]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = autoencoder_conv.fit(train_dataset, 
                    epochs = 10)#0, 
                    #steps_per_epoch = 50)
                    #callbacks=[tensorboard_callback])

Epoch 1/10
500/500 [==============================] - 28s 55ms/step - loss: 0.0064 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.0498
Epoch 2/10
500/500 [==============================] - 27s 53ms/step - loss: 0.0051 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.1031
Epoch 3/10
500/500 [==============================] - 27s 53ms/step - loss: 0.0047 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.1310
Epoch 4/10
500/500 [==============================] - 27s 53ms/step - loss: 0.0045 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.1449
Epoch 5/10
500/500 [==============================] - 27s 53ms/step - loss: 0.0043 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.1539
Epoch 6/10
500/500 [==============================] - 27s 53ms/step - loss: 0.0042 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.1573
Epoch 7/10
500/500 [==============================] - 27s 53ms/step - loss: 0.0041 - accuracy: 0.0000e+00 - kullback_leibler_divergence:

In [15]:
train_dataset

<BatchDataset shapes: ((None, 64, 64, 2), (None, 64, 64, 1)), types: (tf.float32, tf.float32)>

In [12]:
train_images[:,:,:,]

<tf.Tensor: id=194186, shape=(5000, 64, 64, 2), dtype=float32, numpy=
array([[[[0.00853305, 0.01355046],
         [0.07561567, 0.08178521],
         [0.06929585, 0.04237674],
         ...,
         [0.02957213, 0.07670554],
         [0.0052856 , 0.02547984],
         [0.00679371, 0.00316351]],

        [[0.02380365, 0.02444831],
         [0.03965709, 0.00854627],
         [0.03794348, 0.07607996],
         ...,
         [0.03148047, 0.03917638],
         [0.05714347, 0.00154726],
         [0.01648956, 0.0026379 ]],

        [[0.06024077, 0.02149139],
         [0.03017522, 0.06895515],
         [0.00516971, 0.03185792],
         ...,
         [0.05884131, 0.07900978],
         [0.0607749 , 0.03281374],
         [0.03484062, 0.08176979]],

        ...,

        [[0.01297466, 0.0638418 ],
         [0.02544382, 0.03456907],
         [0.00232573, 0.03444039],
         ...,
         [0.05761433, 0.08983523],
         [0.02710241, 0.03357909],
         [0.08884429, 0.096869  ]],

        [[0.

In [13]:
train_images[:,:,:,1:]

<tf.Tensor: id=194191, shape=(5000, 64, 64, 1), dtype=float32, numpy=
array([[[[0.01355046],
         [0.08178521],
         [0.04237674],
         ...,
         [0.07670554],
         [0.02547984],
         [0.00316351]],

        [[0.02444831],
         [0.00854627],
         [0.07607996],
         ...,
         [0.03917638],
         [0.00154726],
         [0.0026379 ]],

        [[0.02149139],
         [0.06895515],
         [0.03185792],
         ...,
         [0.07900978],
         [0.03281374],
         [0.08176979]],

        ...,

        [[0.0638418 ],
         [0.03456907],
         [0.03444039],
         ...,
         [0.08983523],
         [0.03357909],
         [0.096869  ]],

        [[0.09753848],
         [0.04908586],
         [0.04849957],
         ...,
         [0.01016187],
         [0.04258858],
         [0.02128114]],

        [[0.02962798],
         [0.00290593],
         [0.06358675],
         ...,
         [0.03390259],
         [0.00764224],
         [0.07482